In [ ]:
%matplotlib ipympl
import matplotlib as mpl
import matplotlib.transforms
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd

import experimental.beacon_sim.ekf_slam_python as esp
import planning.probabilistic_road_map_python as prmp
from experimental.beacon_sim.ekf_slam_estimate_pb2 import EkfSlamEstimate as EstimateProto
from experimental.beacon_sim.mapped_landmarks_pb2 import MappedLandmarks
import common.liegroups.se2_python as se2
import common.liegroups.so2_python as so2

import itertools
import embag_python as embag
import enum
from typing import NamedTuple


mpl.style.use('ggplot')

In [ ]:
bag_file_dir = Path('/home/erick/Dropbox (MIT)/brm_bag_files/gt_estimator')
raw_bag_file_dir = Path('/home/erick/Dropbox (MIT)/brm_bag_files/raw')
road_map_path = Path('/home/erick/Dropbox (MIT)/brm_bag_files/killian_court_road_map.pb')
map_path = Path('/home/erick/Dropbox (MIT)/brm_bag_files/killian_court_map.pb')

In [ ]:
baseline_bags = sorted(list(bag_file_dir.glob('*baseline*present*[0-9].bag')))
brule_bags = sorted(list(bag_file_dir.glob('*brule*present*[0-9].bag')))
ONE_HUNDRED_MB = 100 * 1024 * 1024
brule_bags = [p for p in brule_bags if p.stat().st_size > ONE_HUNDRED_MB]

baseline_bags = [('206_present' in p.name, p) for p in baseline_bags]
brule_bags = [('206_present' in p.name, p) for p in brule_bags]

In [ ]:
brule_bags

In [ ]:
baseline_bags

In [ ]:
road_map = prmp.RoadMap.from_proto_string(road_map_path.read_bytes())
map_proto = MappedLandmarks()
_ = map_proto.ParseFromString(map_path.read_bytes())

In [ ]:
def read_gt_bag(path: Path):
    bag = embag.Bag(path)
    map_from_robot_at_time = []
    for topic, msg, time in bag.read_messages(topics=["/map"]):
        proto_string = bytes(msg.estimate_proto)
        estimate = esp.EkfSlamEstimate.from_proto_string(proto_string)
        map_from_robot_at_time.append((estimate.time_of_validity, estimate.local_from_robot()))
    pos = np.stack([mfr.translation() for _, mfr in map_from_robot_at_time])
    return pos



In [ ]:
baseline_positions = [read_gt_bag(str(x[1])) for x in baseline_bags]
brule_positions = [read_gt_bag(str(x[1])) for x in brule_bags]

In [ ]:
def plot_road_map(road_map, show_text=True):
    pts = np.stack(road_map.points())

    if show_text:
        for i, pt in enumerate(pts):
            label = str(i)
            if i == 16:
                label = 'Start'
            elif i == 17:
                label = "Goal"
            plt.text(*pt, label, c='white', backgroundcolor=(0.0, 0.0 , 0.0, 0.2))

    if road_map.has_start_goal():
        for idx, marker in [(road_map.START_IDX, 'rs'), (road_map.GOAL_IDX, 'y*')]:
            pt = road_map.point(idx)
            plt.plot(*pt, marker)
            

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])

    if road_map.has_start_goal():
        for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
            pt = road_map.point(idx)
            neighbors = road_map.neighbors(idx)
            for _, neighbor_pt in neighbors:
                line_segments.append([pt, neighbor_pt])
    
    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0), label='Road Map Edge')
    ax = plt.gca()
    ax.add_collection(edges)

    plt.plot(pts[:, 0], pts[:, 1], '*', label='Road Map Node')

def plot_mapped_landmarks(mapped_landmarks):
    ids = mapped_landmarks.beacon_ids
    xs = []
    ys = []
    for i, beacon_in_local in enumerate(mapped_landmarks.beacon_in_local):
        beacon_in_map = mapped_landmarks.beacon_in_local[i]
        xs.append(beacon_in_map.data[0])
        ys.append(beacon_in_map.data[1])
        # plt.text(beacon_in_map.data[0], beacon_in_map.data[1], f't_{ids[i]}')
    plt.plot(xs, ys, 'o', label='Landmark', markersize=3)

def plot_background(image):
    figure_from_data = plt.gca().transData
    image_from_figure = mpl.transforms.Affine2D().scale(0.09).translate(-15, -60.0).rotate_deg_around(0, 0, -14.0).translate(0.0, 5.0)

    im = plt.gca().imshow(image, origin='lower')
    transform = im.get_transform()
    # print(im.get_transform())
    im.set_transform(image_from_figure + figure_from_data)

In [ ]:
kresge_background = plt.imread('/home/erick/Dropbox (MIT)/brm_bag_files/kresge_background.png')
kresge_background = np.flip(kresge_background, 0)

In [ ]:

fig = plt.figure()

plot_road_map(road_map)
plot_mapped_landmarks(map_proto)
plot_background(kresge_background)

plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.legend(loc='center', bbox_to_anchor=(0.95,0.95))

plt.axis('equal')
plt.xlim(-5, 65)
plt.ylim(-45, 10)

# plt.ylim(-80, 30)
plt.title('Road Map and Landmarks')
# plt.tight_layout()

In [ ]:


plt.figure(figsize=(8, 4), dpi=100)
ax = plt.subplot(121)
plot_background(kresge_background)
plot_road_map(road_map)
plot_mapped_landmarks(map_proto)
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.legend(loc='center', bbox_to_anchor=(0.6,0.1))
# plt.axis('equal')
plt.ylim(-50, 10)
plt.xlim(-5, 65)
plt.grid(visible=False)


plt.title('Road Map and Landmarks')

plt.subplot(122, sharex=ax, sharey=ax)
plot_background(kresge_background)
plot_road_map(road_map, show_text=False)


for pos in baseline_positions:
    baseline_artist = plt.plot(pos[:, 0], pos[:, 1], label='BRULE-E', color='tab:orange')

for pos in brule_positions:
    brule_artist = plt.plot(pos[:, 0], pos[:, 1], label='BRULE', color='tab:blue')

plt.legend(handles=[baseline_artist[0], brule_artist[0]])

plt.xlabel('X (m)')
plt.title('Trajectories')
plt.grid(visible=False)

# plt.subplot(133, sharex=ax, sharey=ax)
# plot_background(kresge_background)
# plot_road_map(road_map, show_text=False)
# for pos in brule_positions:
#     plt.plot(pos[:, 0], pos[:, 1])

# plt.xlabel('X (m)')
# plt.title('BRULE Trajectories')
# plt.grid(visible=False)
plt.tight_layout()
plt.savefig('/tmp/trajectories.svg')

In [ ]:

class PointSource(enum.Enum):
    true = 1
    nominal = 2

class Point(NamedTuple):
    idx: int
    source: PointSource

def sample_path_edge(road_map, start_idx, end_idx, max_ds_m = 1.0):
    start_pos = road_map.point(start_idx)
    end_pos = road_map.point(end_idx)
    d_pos = end_pos - start_pos

    edge_heading_rad = np.arctan2(d_pos[1], d_pos[0])
    
    edge_length_m = np.linalg.norm(d_pos)
    num_steps = int((edge_length_m + 0.5 * max_ds_m) / max_ds_m)

    poses = []
    distances = []
    for i in range(0, num_steps+1):
        frac = i / num_steps
        local_from_robot_pos = start_pos + frac * d_pos
        poses.append(se2.SE2(edge_heading_rad, local_from_robot_pos))
        distances.append(frac * edge_length_m)
    return poses, distances
        

def sample_node_path(road_map, node_path, max_ds_m = 1.0):
    poses = []
    distances = []
    node_distances = [0.0]
    for start, end in zip(node_path[:-1], node_path[1:]):
        edge_poses, edge_distances = sample_path_edge(road_map, start, end, max_ds_m)
        if not poses:
            poses.extend(edge_poses)
            distances.extend(edge_distances)
        else:
            poses.extend(edge_poses[1:])
            offset_dist_m = distances[-1]
            distances.extend([d + offset_dist_m for d in edge_distances[1:]])
        node_distances.append(distances[-1])
    return poses, np.array(distances), np.array(node_distances)

def find_nearest(q, polyline):
    delta = polyline - q
    norms = np.linalg.norm(delta, axis=1)
    return np.argmin(norms, keepdims=True)
    

def find_correspondence(positions, nominal_positions):
    D = {}
    unmatched_nom_idxs = set(range(len(nominal_positions)))
    for true_idx, true_pt in enumerate(positions):
        nearest_on_nominal_idxs = find_nearest(true_pt, nominal_positions)
        for nom_idx in nearest_on_nominal_idxs:
            nom_key = Point(nom_idx, PointSource.nominal)
            nearest_pts = D.get(nom_key, [])
            nearest_pts.append(Point(true_idx, PointSource.true))
            D[nom_key] = nearest_pts
            unmatched_nom_idxs -= {nom_idx}
        
    for nom_idx in unmatched_nom_idxs:
        nom_key = Point(nom_idx, PointSource.nominal)
        nom_pt = nominal_positions[nom_idx]
        true_idxs = find_nearest(nom_pt, positions)
        for true_idx in true_idxs:
            true_key = Point(true_idx, PointSource.true)
            if true_key in D:
                D[true_key].append(nom_key)
            else:
                nom_closest_to_true_idx = find_nearest(positions[true_idx], nominal_positions)
                assert len(nom_closest_to_true_idx) == 1
                
                nom_closest_to_true_key = Point(nom_closest_to_true_idx[0], PointSource.nominal)
                if nom_closest_to_true_key in D:
                    D[nom_closest_to_true_key].remove(true_key)
                    D[true_key] = [nom_key]
                else:
                    del D[nom_closest_to_true_key]
                    D[true_key] = [nom_key, Point(nom_closest_to_true_idx, PointSource.nominal)]

    partition = []
    for nom_idx in range(len(nominal_positions)):
        nom_key = Point(nom_idx, PointSource.nominal)
        if nom_key in D:
            partition.append((D[nom_key], [nom_key]))
    for true_idx in range(len(positions)):
        true_key = Point(true_idx, PointSource.true)
        if true_key in D:
            partition.append(([true_key], D[true_key]))
    
    return partition

def compute_minimal_error(positions, map_from_nominal_poses, correspondence):
    errors = []
    for true_idxs, nominal_idxs in correspondence:
        assert len(nominal_idxs) == 1
        
        nominal_idx = nominal_idxs[0].idx
        map_from_nominal_robot = map_from_nominal_poses[nominal_idx]
        true_idxs = [x.idx for x in true_idxs]
        nearest_true_idx_in_set = find_nearest(map_from_nominal_robot.translation(), positions[true_idxs])
        nearest_true_idx = true_idxs[nearest_true_idx_in_set[0]]
        nearest_true_pt = positions[nearest_true_idx]
        errors.append(map_from_nominal_robot.inverse() @ nearest_true_pt.T)
    return np.hstack(errors).T

def compute_crosstrack_error(positions, nominal_poses):
    # Sample the node path 
    nominal_positions = np.vstack([x.translation() for x in nominal_poses])
    
    # Find a correspondence between the positions and the sampled positions
    correspondence = find_correspondence(positions, nominal_positions)

    # For each point on the nominal path, compute the minimum error
    errors = compute_minimal_error(positions, nominal_poses, correspondence)
    
    # Compute the position error
    return correspondence, errors


In [ ]:
def make_crosstrack_error_plot(config_and_positions, node_path, road_map):
    errors = []
    configs = []
    nominal_poses, geodesic_distances, node_distances = sample_node_path(road_map, node_path)

    
    for i, (node_distance, node_id) in enumerate(zip(node_distances[1:-1], node_path[1:-1])):
        label = 'Node Id' if i == 0 else ''
        plt.axvline(node_distance, color='k', linestyle='dashed', linewidth=0.5, label=label)
        plt.text(node_distance + 1, 1.25, node_id)

    
    for config, positions in config_and_positions:
        
        correspondence, error = compute_crosstrack_error(positions, nominal_poses)
        errors.append(error)
        configs.append(config)

    configs_labeled = set()
    for config, error in zip(configs, errors):
        label = 'Landmark 6 Present' if config else 'Landmark 9 Present'
        if config in configs_labeled:
            label = ''
        else:
            configs_labeled.add(config)
        plt.plot(geodesic_distances, error[:, 1], 'tab:blue' if config else 'tab:orange', linewidth=0.5, label=label)

    errors = np.dstack(errors)
    mean_error = np.mean(errors, axis=2)
    std_error = np.std(errors, axis=2)

    plt.plot(geodesic_distances, mean_error[:, 1], 'k', label='Mean Error')
    plt.fill_between(geodesic_distances,
                    mean_error[:, 1]-2*std_error[:, 1],
                    mean_error[:, 1]+2*std_error[:, 1],
                    color='tab:green',
                    alpha=0.25, label = '$2\sigma$')
    
    xlim = plt.xlim()
    plt.xticks(np.arange(0, 100, 15))
    plt.xlim(xlim)
    plt.ylim(-1.5, 1.5)

    ...

In [ ]:
plt.figure(figsize=(8, 4), dpi=100)
plt.subplot(121)
baseline_path = [16, 0, 5, 6, 10, 15, 17]

baseline_config_and_positions = [(config, position) for (config, _), position in zip(baseline_bags, baseline_positions)]
make_crosstrack_error_plot(baseline_config_and_positions, baseline_path, road_map)
plt.title('BRULE-E Crosstrack Error')
plt.xlabel('Distance along path (m)')
plt.ylabel('Crosstrack Error (m)')

plt.legend(loc='lower left')

plt.subplot(122)
brule_path = [16, 0, 1, 6, 9, 14, 15, 17]
brule_config_and_positions = [(config, position) for (config, _), position in zip(brule_bags, brule_positions)]
make_crosstrack_error_plot(brule_config_and_positions, brule_path, road_map)
plt.xlabel('Distance along path (m)')



plt.title('BRULE Crosstrack Error')
plt.tight_layout()
plt.savefig('/tmp/crosstrack_error.svg')
# Consider including mean error line and std deviation areas, thinner lines for actual trajectories
# marker to see when observations are made
# Add table with end position error


In [ ]:
baseline_final_positions = np.stack([pos[-1] for _, pos in baseline_config_and_positions])
brule_final_positions = np.stack([pos[-1] for _, pos in brule_config_and_positions])
goal_pos = road_map.point(17)

In [ ]:
baseline_final_positions = pd.DataFrame(baseline_final_positions, columns=['x', 'y'])
brule_final_positions = pd.DataFrame(brule_final_positions, columns=['x', 'y'])
baseline_final_positions['Approach'] = 'BRULE-E'
brule_final_positions['Approach'] = 'BRULE'

final_positions = pd.concat([baseline_final_positions, brule_final_positions])
    

In [ ]:
statistics = {}
for approach, df in final_positions.groupby(by='Approach'):
    component_error = df[['x', 'y']] - goal_pos
    distance_error = np.sqrt(component_error['x']**2 + component_error['y']**2)
    mean_error = distance_error.mean()
    std_error = distance_error.std()
    # print(approach, mean_error, std_error)
    statistics[approach] = {
        'Mean Error (m)': mean_error,
        'Std Dev (m)': std_error
    }
statistics = pd.DataFrame(statistics).transpose()

In [ ]:
import pandas.io.formats.style
styler = pd.io.formats.style.Styler(statistics)
styler.format(precision=3)

In [ ]:
print(styler.to_latex(column_format='lrr', hrules=True))